In [1]:
# import sys
# print(sys.executable)


In [2]:
# import platform
# print(platform.architecture())


In [3]:
# import jpype
# jpype.startJVM()  # Startet die JVM
# print("JVM erfolgreich gestartet!")
# jpype.shutdownJVM()


In [4]:
# from jpmml_evaluator import make_evaluator

# # Teste, ob die JVM erfolgreich gestartet werden kann
# try:
#     evaluator = make_evaluator('../models/analyzed_model.pmml')  # Verwende einen einfachen PMML-Pfad
#     evaluator.verify()
#     print("JVM erfolgreich gestartet und PMML-Modell erfolgreich geladen.")
# except Exception as e:
#     print(f"Fehler: {e}")


In [5]:
# print("START")
# import os
# import pandas as pd
# from pypmml import Model

# # Pfad zur PMML-Datei und zur CSV-Datei
# pmml_file_path = '/mnt/data/analyzed_model.pmml'
# csv_file_path = 'models/generated_data.csv'  # Passe den Pfad zur CSV-Datei bei Bedarf an

# # Funktion zum Laden des Modells mit verbesserter Fehlerbehandlung
# def load_model(file_path):
#     try:
#         # Modell mit pypmml laden
#         loaded_model = Model.load(file_path)
#         if loaded_model is not None:
#             print("Modell erfolgreich geladen.")
#             return loaded_model
#     except Exception as e:
#         print(f"Fehler beim Laden des PMML-Modells: {e}")
#     return None

# # Modell laden
# loaded_model = load_model(pmml_file_path)

# if loaded_model is not None:
#     # Lade die generierten Daten
#     data = pd.read_csv(csv_file_path)
    
#     # Vorverarbeitung: Sicherstellen, dass kategorische Variablen als solche erkannt werden
#     data['Geschlecht'] = data['Geschlecht'].astype('category')
#     data['Kategorie'] = data['Kategorie'].astype('category')

#     # Vorhersagen durchführen
#     predictions = loaded_model.predict(data)
#     print(predictions)  # Ergebnisse anzeigen
    
# else:
#     print("Das Modell konnte nicht geladen werden.")


In [6]:
# import pandas as pd
# from sklearn.tree import DecisionTreeRegressor  # Verwende DecisionTreeRegressor
# from sklearn2pmml.pipeline import PMMLPipeline
# from sklearn2pmml import sklearn2pmml
# from jpmml_evaluator import make_evaluator

# # PMML-Dateipfad und CSV-Dateipfad
# pmml_file_path = '../models/analyzed_model.pmml'
# csv_file_path = '../models/generated_data.csv'

# # Schritt 1: Modell erstellen, trainieren und als PMML speichern
# def train_and_save_model():
#     data = pd.read_csv(csv_file_path)
#     X = data.drop(columns=['Zielwert'])  # Ersetze 'Zielwert' mit dem tatsächlichen Zielnamen
#     y = data['Zielwert']  # Ersetze 'Zielwert' mit dem tatsächlichen Zielnamen

#     # Umwandeln der kategorischen Variablen in Dummy-Variablen
#     X = pd.get_dummies(X)

#     # Pipeline erstellen und trainieren
#     pipeline = PMMLPipeline([
#         ("regressor", DecisionTreeRegressor())  # Verwende DecisionTreeRegressor
#     ])
#     pipeline.fit(X, y)

#     # Modell in PMML-Datei speichern
#     sklearn2pmml(pipeline, pmml_file_path, with_repr=True)
#     print(f"Modell erfolgreich gespeichert als {pmml_file_path}.")

# # Schritt 2: Modell laden
# def load_model(file_path):
#     try:
#         evaluator = make_evaluator(file_path)
#         evaluator.verify()  # Überprüfen des Modells
#         print("Modell erfolgreich geladen mit jpmml-evaluator.")
#         return evaluator
#     except Exception as e:
#         print(f"Fehler beim Laden des PMML-Modells mit jpmml-evaluator: {e}")
#     return None

# # Schritt 3: Modell verwenden, um Vorhersagen zu machen
# def make_predictions(evaluator):
#     data = pd.read_csv(csv_file_path)
#     # Umwandeln der kategorischen Variablen in Dummy-Variablen
#     data = pd.get_dummies(data)

#     # Vorhersagen durchführen
#     predictions = evaluator.evaluate(data.to_dict(orient='records'))  # Daten ins Dictionary umwandeln

#     # Die Ergebnisse hier direkt verwenden
#     print("Vorhersagen:", predictions)  # Vorhersagen ausgeben

# # Hauptlogik
# if __name__ == "__main__":
#     train_and_save_model()  # Trainiere und speichere das Modell
#     loaded_model = load_model(pmml_file_path)  # Lade das Modell

#     if loaded_model is not None:
#         make_predictions(loaded_model)  # Mache Vorhersagen
#     else:
#         print("Das Modell konnte nicht geladen werden.")


In [ ]:
import os
import numpy as np
import pandas as pd
from jpmml_evaluator import make_evaluator
from xgrove import grove  # Importiere die Grove-Klasse aus dem xgrove Paket
from sklearn.preprocessing import StandardScaler

# Setze den Pfad für Graphviz
os.environ["PATH"] += os.pathsep + 'C:/Program Files/Graphviz/bin/'

def debug_model_loading(pmml_file_path):
    """Versucht, das PMML-Modell zu laden und zu validieren."""
    try:
        evaluator = make_evaluator(pmml_file_path)
        evaluator.verify()
        print("PMML-Modell erfolgreich geladen und verifiziert.")
        return evaluator
    except Exception as e:
        print(f"Fehler beim Laden des PMML-Modells: {e}")
        return None

def debug_data_loading(csv_file_path):
    """Lädt die CSV-Daten und gibt eine Warnung, falls NaN-Werte vorhanden sind."""
    try:
        data = pd.read_csv(csv_file_path)
        print(f"Daten erfolgreich geladen: {data.head()}")
        
        # Überprüfen auf NaN-Werte und leere Strings
        print(f"NaN-Werte in den Daten: {data.isna().sum()}")
        print(f"Leere Strings in den Daten: {(data == '').sum()}")
        
        # Entferne NaN-Werte
        if data.isna().sum().sum() > 0:
            print("Warnung: Es gibt NaN-Werte in den Eingabedaten. Diese werden entfernt.")
            data = data.dropna()
        
        return data
    except Exception as e:
        print(f"Fehler beim Laden der CSV-Datei: {e}")
        return None

def preprocess_data(data):
    """Bereitet die Eingabedaten vor, indem sie skaliert werden, wenn nötig."""
    # Überprüfen auf NaN-Werte nach Entfernen
    print(f"Datenform nach NaN-Entfernung: {data.shape}")

    # Skalierung der Daten, falls notwendig
    scaler = StandardScaler()
    data_scaled = data.copy()
    data_scaled['x'] = scaler.fit_transform(data[['x']])

    return data_scaled

def load_and_evaluate_model(pmml_file_path, csv_file_path):
    """Lädt das PMML-Modell und erstellt ein Surrogat-Modell mit xgrove."""
    
    # 1. Lade die Trainingsdaten
    data = debug_data_loading(csv_file_path)
    if data is None:
        return

    # 2. Preprocessiere die Daten
    data = preprocess_data(data)

    # 3. Lade das PMML-Modell
    evaluator = debug_model_loading(pmml_file_path)
    if evaluator is None:
        return

    # 4. Generiere Surrogat-Zielwerte für die Daten
    surrogate_targets = []
    for i, row in data.iterrows():
        input_data = row.to_dict()  # Erstelle ein Dictionary für eine Zeile
        prediction = evaluator.evaluate(input_data)  # Einzeldatensatz bewerten
        surrogate_targets.append(prediction.get('Predicted_y', None))  # Nutze den Rückgabeschlüssel des Modells
        if i % 10 == 0:  # Debugging: Zeige alle 10 Zeilen
            print(f"Verarbeite Zeile {i+1}/{len(data)}")

    # 5. Überprüfe auf NaN-Werte in den Surrogat-Zielwerten
    print("Überprüfe auf NaN-Werte in den Surrogat-Zielwerten...")
    surrogate_targets = [val for val in surrogate_targets if val is not None]  # Entferne NaN-Werte
    print(f"Anzahl der gültigen Surrogat-Zielwerte: {len(surrogate_targets)}")

    # Wenn nach der Bereinigung keine Surrogat-Zielwerte übrig sind, einen Fehler werfen
    if not surrogate_targets:
        raise ValueError("Fehler beim Erstellen der Surrogat-Zielwerte: Keine gültigen Zielwerte vorhanden.")
    
    # Surrogat-Targets in den DataFrame aufnehmen
    data['surrogatetarget'] = surrogate_targets
    print(f"Surrogat-Zielwerte wurden erfolgreich zum Datensatz hinzugefügt. Datenform: {data.shape}")

    # 6. Instanziiere die Grove-Klasse mit Surrogat-Targets und dem Modell
    grove_model = grove(model=evaluator, data=data, trained=True)
    
    # Berechnung des Surrogat-Modells durchführen
    grove_model.calculateGrove()

    print("Berechnungen abgeschlossen.")
    print(grove_model.get_result())

    # 7. Zusätzliche Modellattribute anzeigen (falls vorhanden)
    model_classes = evaluator.getModelClasses()  # Modellklassen, falls definiert
    print(f"Model Klassen: {model_classes}")

if __name__ == "__main__":
    # Definiere die Datei- und CSV-Pfade
    pmml_file_path = '../models/linear_model.pmml'
    csv_file_path = '../models/generated_data.csv'

    # Führe das Modell aus
    load_and_evaluate_model(pmml_file_path, csv_file_path)


c:\Users\jjacq\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


Berechnungen abgeschlossen.
[   trees  rules   upsilon       cor
0      4     10  0.846721  0.924040
1      8     11  0.923380  0.962863
2     16     11  0.964851  0.982647
3     32     11  0.988227  0.994175
4     64     11  0.998464  0.999233
5    128     11  0.999977  0.999989, [    variable upper_bound_left  \
0   Interept              NaN   
1  feature_0                    
2  feature_1                    
3  feature_2                    
4  feature_3                    
5  feature_4                    
6  feature_5                    
7  feature_6                    
8  feature_7                    
9  feature_9                    

                                         levels_left  \
0                                                NaN   
1  [ |   | 1 | . | 0 | 2 | 4 | 0 | 6 | 2 | 5 | 3 ...   
2  [ |   | 2 | . | 0 | 6 | 1 | 5 | 0 | 3 | 5 | 8 ...   
3  [ |   | 2 | . | 5 | 5 | 8 | 1 | 9 | 9 | 2 | 9 ...   
4  [ | - | 5 | . | 6 | 4 | 2 | 4 | 7 | 5 | 9 | 9 ...   
5  [ |   | 1 | . 

c:\Users\jjacq\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\ensemble\_gb.py:668: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)  # TODO: Is this still required?


ValueError: Input y contains NaN.